# Hands-on Practice: Visualizing Attention in Language Models
<br>

<a href="https://colab.research.google.com/github/seongmin-mun/Courses/blob/master/2025/20250906_CUFS/Code_CUFS_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Step 1. 환경 설정하기

In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: NVIDIA A100-SXM4-40GB


In [2]:
!pip install transformers==4.20.1
!pip install tensorflow
!pip install keras_preprocessing
!pip install keras
!pip install keras_preprocessing
!pip install sentencepiece
!pip install bertviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 69.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.5/157.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Step 2. 모델 불러오기
- https://huggingface.co/simonmun

In [3]:
from transformers import GPT2Model, GPT2Tokenizer
from bertviz.transformers_neuron_view import GPT2Model, GPT2Tokenizer
from transformers import utils
from bertviz import model_view
from transformers import AutoTokenizer, AutoModelForSequenceClassification

utils.logging.set_verbosity_error()

model_name = "simonmun/LoclassificationGPT2"
model = AutoModelForSequenceClassification.from_pretrained(model_name, output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

# Step 3.1. 모델 사용해서 '으로'의 의미 분류하기

In [4]:
def outresult(guess):
    guess = int(guess)
    outClass = ""
    if guess == 0:
        outClass = "FNS"
    elif guess == 1:
        outClass = "INS"
    elif guess == 2:
        outClass = "DIR"
    elif guess == 3:
        outClass = "EFF"
    elif guess == 4:
        outClass = "CRT"
    elif guess == 5:
        outClass = "LOC"

    return outClass

import torch

inputs = ["안정환이 선발로 나왔다.","케이크를 칼로 잘랐다.","길이 사고로 인해 막혔다."]

for each in inputs:

  input = tokenizer(each, return_tensors="pt")

  with torch.no_grad():
      logits = model(**input).logits

      print(logits)
      prediction_labels = logits.argmax(axis=-1).flatten().tolist()
      guess = prediction_labels[0]
      print("입력하신 문장,","'"+each+"'에서 조사 '으로'의 기능은 ",outresult(guess),"입니다.")

tensor([[18.2563, -8.5897, -1.2391, -4.4538, -1.0930, -3.6569]])
입력하신 문장, '안정환이 선발로 나왔다.'에서 조사 '으로'의 기능은  FNS 입니다.
tensor([[-2.9609, 16.2467, -3.2532, -2.1722, -4.3745, -2.3675]])
입력하신 문장, '케이크를 칼로 잘랐다.'에서 조사 '으로'의 기능은  INS 입니다.
tensor([[  7.5398, -12.3607,  -6.4936,  16.2473,   0.5460,  -7.5865]])
입력하신 문장, '길이 사고로 인해 막혔다.'에서 조사 '으로'의 기능은  EFF 입니다.


# Step 3.2. 모델 사용해서 인공지능 모델의 Attention 시각화하기

In [5]:
inputs = tokenizer.encode("경북대학교를 학회 개최지로 정하였다.", return_tensors='pt')
outputs = model(inputs)
attention = outputs[-1]
tokens = tokenizer.convert_ids_to_tokens(inputs[0])

from bertviz import head_view
head_view(attention, tokens)

<IPython.core.display.Javascript object>